In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

In [2]:
import geopandas as gpd
import pandas as pd
from siuba import *
import numpy as np

from segment_speed_utils import helpers, gtfs_schedule_wrangling
from shared_utils import rt_utils

import sys
sys.path.append('./scripts/')

from calitp_data_analysis import geography_utils

In [3]:
from update_vars import (analysis_date, EXPORT_PATH, GCS_FILE_PATH, PROJECT_CRS,
SEGMENT_BUFFER_METERS, HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD, SHARED_STOP_THRESHOLD,
                        HALF_MILE_BUFFER_METERS, INTERSECTION_BUFFER_METERS)

In [4]:
import prep_pairwise_intersections

In [5]:
# !pip install calitp-data-analysis==2025.6.24
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
gcsgp = GCSGeoPandas()

In [6]:
import pyproj
geodesic = pyproj.Geod(ellps="WGS84")

## Refine intersections with azimuth

In [7]:
corridors = prep_pairwise_intersections.prep_bus_corridors(is_ms_precursor=True)

In [8]:
# corridors.columns

In [9]:
# segments = gcsgp.read_parquet(f'{GCS_FILE_PATH}hqta_segments.parquet')

In [10]:
# rail = gcsgp.read_parquet(f'{GCS_FILE_PATH}rail_brt_ferry.parquet')

In [11]:
# segments = rt_utils.add_origin_destination(segments.to_crs(geography_utils.WGS84))

In [12]:
def add_azimuth(row: pd.Series, geodesic: pyproj.Geod) -> pd.Series:
    '''
    Given a row of a GeoDataFrame with origin and destination points in WGS84,
    find forward and back azimuth. 
    pyproj method returns signed azimuth -180(south) - -90(west) - 0(north) - +90(east) - +180(south),
    convert this to degrees 0-360 and keep the higher one to enable consistent comparison later
    '''
    # print(geodesic)
    long1, lat1 = row.origin.coords[0]
    long2, lat2 = row.destination.coords[0]
    assert all([abs(x) <= 180 for x in [long1, lat1, long2, lat2]]), 'CRS must be WGS84'
    fwd_azimuth, back_azimuth, _distance = geodesic.inv(long1, lat1, long2, lat2, return_back_azimuth=True)
    # signed_azimuth_to_360_deg = lambda x: 360 + x if x <= 0 else x
    # fwd_azimuth = signed_azimuth_to_360_deg(fwd_azimuth)
    # back_azimuth = signed_azimuth_to_360_deg(back_azimuth)
    max_azimuth = max(fwd_azimuth, back_azimuth)
    row['fwd_azimuth'] = fwd_azimuth
    row['back_azimuth'] = back_azimuth
    row['max_azimuth'] = max_azimuth
    return row

In [13]:
# with_azi = segments.apply(add_azimuth, axis=1, geodesic=geodesic) #  added to segments script

In [14]:
signed_azimuth_to_360_deg = lambda x: 360 + x if x < 0 else x

In [15]:
signed_azimuth_to_360_deg(-179)

181

In [16]:
signed_azimuth_to_360_deg(179)

179

In [17]:
corridors.head(3)

,schedule_gtfs_dataset_key,route_id,route_key,segment_sequence,geometry,hqta_segment_id,segment_direction,fwd_azimuth,fwd_azimuth_360,route_direction,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,route_dir_count,hq_transit_corr,ms_precursor,route_type
0,c499f905e33929a641f083dad55c521e,57,7cbc9d29878e56796e32ec197512a38f,15,"POLYGON ((-201535.497 -18383.876, -201536.496 ...",1200119,north-south,-13.775891,346.224109,east-west,51697,12,57_1,16.0,4.00,4.0,1,True,True,3
1,0666caf3ec1ecc96b74f4477ee4bc939,501-13191,44e5c016a53088e305d7d37c5f4f81c4,38,"POLYGON ((170060.631 -427890.378, 170070.545 -...",1218022,east-west,70.512751,70.512751,east-west,1866,13,260-13191_0,18.0,4.33,4.5,1,True,True,3
2,aea4108997c66a74fbdae27b34b69fde,130,6c056aa1e5625b7042bfbefae2c3c39b,1,"POLYGON ((-212375.273 -23941.486, -212345.959 ...",1331405,east-west,-125.735122,234.264878,north-south,42209,10,101_0,14.0,3.33,3.5,3,False,True,3


In [18]:
import altair as alt
# alt.data_transformers.enable("vegafusion")
data = corridors[['fwd_azimuth_360', 'segment_direction']].sample(4999) #  sample to avoid needing vegafusion for this visual
alt.Chart(data, title = 'HQTA Segment Directions and Azimuth').mark_bar().encode(
    # x=alt.X('azimuth'),
    x=alt.X('fwd_azimuth_360', bin=alt.Bin(step=5)),
    y='count()',
    color='segment_direction'
)

alt.Chart(...)

In [19]:
# corridors.explore(column='fwd_azimuth_360')

In [121]:
back_azi = lambda x: x - 180 if x >= 180 else x + 180

def azimuth_360_compare(azi1, azi2) -> float:
    '''
    compare two 360-degree azimuths
    '''
    if azi1 >= azi2:
        return azi1 - azi2
    else:
        return azi2 - azi1

def find_intersections_azimuth(azi1, azi2, threshold_degrees = 45) -> bool:
    '''
    With two 360-degree azimuths, compare all combininations of forward
    and back azimuths to see if all are more than a specified degree threshold apart.
    
    find_intersections_azimuth(360, 45) should return True
    find_intersections_azimuth(40, 80) should return False
    '''
    back_azi_2 = back_azi(azi2)
    back_azi_1 = back_azi(azi1)
    to_compare = [(azi1, azi2), (azi1, back_azi_2), (back_azi_1, azi2), (back_azi_1, back_azi_2)]
    compare_all = [azimuth_360_compare(x, y) for x, y in to_compare]
    # print(compare_all)
    return not(any([x < threshold_degrees for x in compare_all]))

In [113]:
find_intersections_azimuth(179, 358)

[179, 1, 1, 181]


False

In [122]:
find_intersections_azimuth(360, 45)

True

In [23]:
find_intersections_azimuth(10, 350)

False

In [24]:
find_intersections_azimuth(350, 0)

False

In [25]:
find_intersections_azimuth(60, 100)

False

In [26]:
find_intersections_azimuth(39, 12)

False

In [115]:
find_intersections_azimuth(45, 359)

True

## test new corridors function

In [28]:
def sjoin_against_other_operators(
    in_group_df: gpd.GeoDataFrame, 
    out_group_df: gpd.GeoDataFrame
) -> pd.DataFrame: 
    """
    Spatial join of the in group vs the out group. 
    This could be the operator vs other operators, 
    or a route vs other routes. This is currently
    east-west vs north-south segments, which requires
    the additional step of excluding intersections
    resulting from the same route changing direction.
    
    Create a crosswalk / pairwise table showing these links.
    
    Compile all of them, because finding intersections is 
    computationally expensive,
    so we want to do it on fewer rows. 
    """
    route_cols = ["hqta_segment_id", "segment_direction", "route_key", "fwd_azimuth_360"]
    
    s1 = gpd.sjoin(
        in_group_df[route_cols + ["geometry"]], 
        out_group_df[route_cols  + ["geometry"]],
        how = "inner",
        predicate = "intersects"
    ).drop(columns = ["index_right", "geometry"])
                
    route_pairs = (
        s1.rename(
            columns = {
                "hqta_segment_id_left": "hqta_segment_id",
                "hqta_segment_id_right": "intersect_hqta_segment_id",
                "fwd_azimuth_360_left": "fwd_azimuth_360",
                "fwd_azimuth_360_right": "intersect_fwd_azimuth_360"
            })
          [["hqta_segment_id", "intersect_hqta_segment_id", "fwd_azimuth_360", "intersect_fwd_azimuth_360"]]
          .drop_duplicates()
          .reset_index(drop=True)
    )
    route_pairs = route_pairs.assign(intersect = route_pairs.apply(
        lambda x: find_intersections_azimuth(x.fwd_azimuth_360, x.intersect_fwd_azimuth_360), axis=1)
        )
    # print(route_pairs.empty)
    return route_pairs.query('intersect')

In [29]:
find_intersections_azimuth(59, 21)

False

In [30]:
test_ecr = sjoin_against_other_operators(corridors.query('route_id == "Rapid 522"'), corridors.query('route_id != "Rapid 522"'))

In [31]:
def pairwise_intersections(
    corridors_gdf: gpd.GeoDataFrame
) -> gpd.GeoDataFrame:
    """
    Do pairwise comparisons of hqta segments.
    Take all the north-south segments and compare to east-west
    and vice versa.
    """
    # Intersect each route with all others
    corridors_gdf = corridors_gdf[corridors_gdf['segment_direction'] != 'inconclusive']
    results = [
        sjoin_against_other_operators(corridors.query('route_key == @route_key'),
                                      corridors.query('route_key != @route_key'))
        for route_key in corridors_gdf.route_key.unique()
    ]
    
    pairs = pd.concat(results, axis=0, ignore_index=True)
    
    segments_p1 = pairs.hqta_segment_id.unique()
    segments_p2 = pairs.intersect_hqta_segment_id.unique()
    
    # Subset the hqta segments that do have hq_transit_corr == True 
    # down to the ones where routes have with sjoin intersections
    corridors2 = (
        corridors_gdf[
            (corridors_gdf.hqta_segment_id.isin(segments_p1)) | 
            (corridors_gdf.hqta_segment_id.isin(segments_p2))]
        .drop_duplicates()
        .sort_values(
            ["schedule_gtfs_dataset_key", "route_id", "hqta_segment_id"], 
            ascending = [True, True, True])
        .reset_index(drop=True)
    )
    
    return pairs, corridors2

In [32]:
pairs, corridors2 = pairwise_intersections(corridors)

/tmp/ipykernel_19780/3929986142.py:19: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  pairs = pd.concat(results, axis=0, ignore_index=True)


In [33]:
import get_intersections

In [34]:
pairs_table = get_intersections.attach_geometry_to_pairs(corridors2, pairs)

In [35]:
results = get_intersections.find_intersections(pairs_table)

In [36]:
results

,geometry,schedule_gtfs_dataset_key,hqta_segment_id
0,"POLYGON ((-224650.541 11039.545, -224639.822 1...",015d67d5b75b5cf2b710bbadadfb75f5,29974336
1,"POLYGON ((-224420.509 10782.263, -224412.799 1...",015d67d5b75b5cf2b710bbadadfb75f5,29974336
2,"POLYGON ((-224685.680 10968.296, -224687.981 1...",015d67d5b75b5cf2b710bbadadfb75f5,66086230
3,"POLYGON ((-224685.680 10968.296, -224687.981 1...",015d67d5b75b5cf2b710bbadadfb75f5,66086230
4,"POLYGON ((-225129.784 12965.510, -225129.067 1...",015d67d5b75b5cf2b710bbadadfb75f5,338746405
...,...,...,...
15549,"POLYGON ((-163036.851 -73591.496, -163032.270 ...",fb467982dcc77a7f9199bebe709bb700,4237653065
15550,"POLYGON ((-163036.851 -73591.496, -163032.270 ...",fb467982dcc77a7f9199bebe709bb700,4237653065
15551,"POLYGON ((-165052.541 -75184.308, -165055.803 ...",fb467982dcc77a7f9199bebe709bb700,4242809268
15552,"POLYGON ((-165277.815 -75230.544, -165276.209 ...",fb467982dcc77a7f9199bebe709bb700,4242809268


### create_bus_hqta_types

In [37]:
import create_bus_hqta_types

In [38]:
def buffer_around_intersections(gdf, buffer_size: int) -> gpd.GeoDataFrame: 
    """
    Draw 500 ft buffers around intersections to better catch stops
    that might fall within it.
    """
#     gdf = gpd.read_parquet(
#         f"{GCS_FILE_PATH}all_intersections.parquet",
#         storage_options={"token": credentials.token}
#     )
    
    gdf = gdf.assign(
        geometry = gdf.geometry.buffer(buffer_size)
    )

    return gdf 

In [39]:
bus_intersections = buffer_around_intersections(results, INTERSECTION_BUFFER_METERS)

In [90]:
# Grab point geom with all stops
trips_cols = ["feed_key", "gtfs_dataset_key"]
gtfs_keys = helpers.import_scheduled_trips(
    analysis_date,
    columns = trips_cols,
    get_pandas=True
)
published_operators_dict = lookback_wrappers.read_published_operators(analysis_date)
print(published_operators_dict)
lookback_trips = lookback_wrappers.get_lookback_trips(published_operators_dict, trips_cols + ['name'])
lookback_trips_ix = lookback_wrappers.lookback_trips_ix(lookback_trips)
gtfs_keys = pd.concat([gtfs_keys, lookback_trips.drop(columns=['name'])])

stops_cols = ["feed_key", "stop_id", "geometry"]
all_stops = helpers.import_scheduled_stops(
    analysis_date,
    get_pandas = True,
    columns = stops_cols,
    crs = PROJECT_CRS
)
lookback_stops = lookback_wrappers.get_lookback_stops(published_operators_dict, lookback_trips_ix, stops_cols,
                                                     crs=PROJECT_CRS)
all_stops = pd.concat([all_stops, lookback_stops])

all_stops = all_stops.merge(
    gtfs_keys,
    on = "feed_key",
).drop(columns = "feed_key")

{'2025-03-12': ['Imperial Valley Transit Schedule', 'Bay Area 511 Rio Vista Delta Breeze Schedule'], '2025-04-16': ['Glenn Schedule'], '2025-05-14': ['LAX FlyAway Schedule', 'Morro Bay Cal-ITP Schedule', 'Bay Area 511 Golden Gate Ferry Schedule', 'Long Beach Schedule', 'Arvin Schedule', 'Bellflower Bus Schedule', 'Sacramento Schedule', 'SBMTD Schedule']}


In [91]:
major_stop_bus_testing = create_bus_hqta_types.create_major_stop_bus(all_stops, bus_intersections)

In [92]:
major_stop_bus_testing

,schedule_gtfs_dataset_key_primary,schedule_gtfs_dataset_key_secondary,stop_id,geometry,hqta_type
0,0bcbcc353367cba8ba4134764abe8ec1,0666caf3ec1ecc96b74f4477ee4bc939,305,POINT (162311.087 -428506.915),major_stop_bus
1,0bcbcc353367cba8ba4134764abe8ec1,0666caf3ec1ecc96b74f4477ee4bc939,280,POINT (162079.540 -428429.657),major_stop_bus
2,0bcbcc353367cba8ba4134764abe8ec1,0666caf3ec1ecc96b74f4477ee4bc939,289,POINT (162332.884 -428538.356),major_stop_bus
3,0bcbcc353367cba8ba4134764abe8ec1,0666caf3ec1ecc96b74f4477ee4bc939,279,POINT (162141.044 -428448.822),major_stop_bus
4,0666caf3ec1ecc96b74f4477ee4bc939,0666caf3ec1ecc96b74f4477ee4bc939,9815,POINT (162101.057 -428426.151),major_stop_bus
...,...,...,...,...,...
13229,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1677,POINT (-121100.959 65968.221),major_stop_bus
13230,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1646,POINT (-121118.783 66010.505),major_stop_bus
13231,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1885,POINT (-124885.624 55396.655),major_stop_bus
13232,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,4022,POINT (-123687.720 51236.099),major_stop_bus


### current pipeline result

In [93]:
major_stop_bus = gcsgp.read_parquet(f'{GCS_FILE_PATH}major_stop_bus.parquet')

In [94]:
major_stop_bus

,schedule_gtfs_dataset_key_primary,schedule_gtfs_dataset_key_secondary,stop_id,geometry,hqta_type
0,0bcbcc353367cba8ba4134764abe8ec1,0666caf3ec1ecc96b74f4477ee4bc939,305,POINT (162311.087 -428506.915),major_stop_bus
1,0bcbcc353367cba8ba4134764abe8ec1,0666caf3ec1ecc96b74f4477ee4bc939,280,POINT (162079.540 -428429.657),major_stop_bus
2,0bcbcc353367cba8ba4134764abe8ec1,0666caf3ec1ecc96b74f4477ee4bc939,289,POINT (162332.884 -428538.356),major_stop_bus
3,0bcbcc353367cba8ba4134764abe8ec1,0666caf3ec1ecc96b74f4477ee4bc939,279,POINT (162141.044 -428448.822),major_stop_bus
4,0666caf3ec1ecc96b74f4477ee4bc939,0666caf3ec1ecc96b74f4477ee4bc939,9815,POINT (162101.057 -428426.151),major_stop_bus
...,...,...,...,...,...
14266,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1677,POINT (-121100.959 65968.221),major_stop_bus
14267,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1646,POINT (-121118.783 66010.505),major_stop_bus
14268,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1885,POINT (-124885.624 55396.655),major_stop_bus
14269,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,4022,POINT (-123687.720 51236.099),major_stop_bus


In [95]:
from shared_utils import webmap_utils

In [96]:
import branca

In [97]:
cmap = branca.colormap.step.Spectral_04.scale(vmin=0, vmax=4)

In [98]:
cmap

In [99]:
major_stop_bus['iteration'] = 0
major_stop_bus_testing['iteration'] = 4

In [100]:
gdf = pd.concat([major_stop_bus, major_stop_bus_testing])

In [101]:
color_state = webmap_utils.set_state_export(gdf, filename='test_azimuth_mts_5', map_title='Azimuth for Major Transit Stops',
                                             cmap=cmap, color_col='iteration')

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [102]:
color_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Azimuth for Major Transit Stops',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/test_azimuth_mts_5.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (34.92180193232379, -119.24705655900215),
  'zoom': 13},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJBemltdXRoIGZvciBNYWpvciBUcmFuc2l0IFN0b3BzIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90ZXN0aW5nL3Rlc3RfYXppbXV0aF9tdHNfNS5nZW9qc29uLmd6IiwgInByb3BlcnRpZXMiOiB7InN0cm9rZWQiOiBmYWxzZSwgImhpZ2hsaWdodF9zYXR1cmF0aW9uX211bHRpcGxpZXIiOiAwLjV9fV0sICJsYXRfbG9uIjogWzM0LjkyMTgwMTkzMjMyMzc5LCAtMTE5LjI0NzA1NjU1OTAwMjE1XSwgInpvb20iOiAxM30='}

In [103]:
major_stop_bus.geometry = major_stop_bus.buffer(HALF_MILE_BUFFER_METERS)
major_stop_bus_testing.geometry = major_stop_bus_testing.buffer(HALF_MILE_BUFFER_METERS)

In [104]:
major_stop_bus.dissolve().geometry.area

0    1.807902e+09
dtype: float64

In [105]:
major_stop_bus_testing.dissolve().geometry.area

0    1.763259e+09
dtype: float64

In [106]:
m = major_stop_bus.dissolve().overlay(major_stop_bus_testing.dissolve(), how='difference').explore(color='red')

In [109]:
# major_stop_bus_testing.dissolve().overlay(major_stop_bus.dissolve(), how='difference').explore(m=m)

## Lookback

* `rail_ferry_brt_stops` trips, stops, st
* `create_hqta_segments` shapes
* `create_aggregate_stop_frequencies` trips, st
* `sjoin_stops_to_segments` stops
* `create_bus_hqta_types` trips, stops
* `assemble_hqta_points` 
* `assemble_hqta_polygons`
    - `assemble_hqta_points.get_agency_info` only

In [65]:
import datetime as dt
import yaml
from shared_utils import catalog_utils, publish_utils
import lookback_wrappers
import assemble_hqta_points
import importlib
importlib.reload(lookback_wrappers)

<module 'lookback_wrappers' from '/home/jovyan/data-analyses/high_quality_transit_areas/lookback_wrappers.py'>

In [66]:
published_operators_dict = lookback_wrappers.read_published_operators(analysis_date)

In [67]:
published_operators_dict

{'2025-03-12': ['Imperial Valley Transit Schedule',
  'Bay Area 511 Rio Vista Delta Breeze Schedule'],
 '2025-04-16': ['Glenn Schedule'],
 '2025-05-14': ['LAX FlyAway Schedule',
  'Morro Bay Cal-ITP Schedule',
  'Bay Area 511 Golden Gate Ferry Schedule',
  'Long Beach Schedule',
  'Arvin Schedule',
  'Bellflower Bus Schedule',
  'Sacramento Schedule',
  'SBMTD Schedule']}

In [68]:
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

In [69]:
GTFS_DATA_DICT.schedule_tables.gtfs_key_crosswalk

'crosswalk/gtfs_key_organization'

In [70]:
GTFS_DATA_DICT.schedule_tables.dir

'gs://calitp-analytics-data/data-analyses/gtfs_schedule/'

In [71]:
trips_cols = [
            "name", "feed_key", "trip_id",
            "route_id", "route_type", "route_desc",
            "gtfs_dataset_key",
        ]

In [72]:
lookback_trips = lookback_wrappers.get_lookback_trips(published_operators_dict, trips_cols)

In [73]:
lookback_trips.head(3)

,name,feed_key,trip_id,route_id,route_type,route_desc,schedule_gtfs_dataset_key,lookback_date
0,Imperial Valley Transit Schedule,cdd0c7f4c8f78f1805af16d34b8ed0c3,010314,1,3,None,fe4aab1717eca5a2935c32c85a35a5bf,2025-03-12
1,Imperial Valley Transit Schedule,cdd0c7f4c8f78f1805af16d34b8ed0c3,010300,1,3,None,fe4aab1717eca5a2935c32c85a35a5bf,2025-03-12
2,Imperial Valley Transit Schedule,cdd0c7f4c8f78f1805af16d34b8ed0c3,010316,1,3,None,fe4aab1717eca5a2935c32c85a35a5bf,2025-03-12


In [74]:
lookback_trips_ix = lookback_wrappers.lookback_trips_ix(lookback_trips)

In [75]:
lookback_trips_ix.head(3)

,name,feed_key,schedule_gtfs_dataset_key,lookback_date
0,Imperial Valley Transit Schedule,cdd0c7f4c8f78f1805af16d34b8ed0c3,fe4aab1717eca5a2935c32c85a35a5bf,2025-03-12
64,Bay Area 511 Rio Vista Delta Breeze Schedule,b289d80491722393827621e2a0286179,a131b466de5793f858b5dec42fbbb4a9,2025-03-12
0,Glenn Schedule,c74eaf4ebabfbb26ac8c9712bbcbb01e,7f2ae5bcb719c5912462858471a28aa5,2025-04-16


In [76]:
st_cols = ["feed_key", "trip_id", "stop_id", "arrival_hour"]

In [77]:
lookback_st = lookback_wrappers.get_lookback_st(published_operators_dict, lookback_trips_ix, st_cols)

In [78]:
lookback_st.head(3)

,feed_key,trip_id,stop_id,arrival_hour
0,cdd0c7f4c8f78f1805af16d34b8ed0c3,010303,53,8.0
1,cdd0c7f4c8f78f1805af16d34b8ed0c3,200305,5,14.0
2,cdd0c7f4c8f78f1805af16d34b8ed0c3,010321,69,20.0


In [79]:
stops_cols = ["feed_key", "stop_id", "stop_name", "geometry"]

In [80]:
lookback_stops = lookback_wrappers.get_lookback_stops(published_operators_dict, lookback_trips_ix, stops_cols)

In [81]:
lookback_stops.head(3)

,feed_key,stop_id,stop_name,geometry
0,b289d80491722393827621e2a0286179,2329722,Walmart,POINT (-173771.189 26928.051)
1,b289d80491722393827621e2a0286179,27694,Main & 2nd,POINT (-148022.773 16896.812)
2,b289d80491722393827621e2a0286179,27695,Hillside Terrace & Flores,POINT (-148388.515 17466.601)


In [82]:
lookback_hqta_shapes = lookback_wrappers.get_lookback_hqta_shapes(published_operators_dict, lookback_trips_ix)

In [83]:
lookback_hqta_shapes.head(3)

,geometry,schedule_gtfs_dataset_key,route_id,direction_id,route_key
2697,"LINESTRING (-147965.518 16828.747, -148025.619...",a131b466de5793f858b5dec42fbbb4a9,50,0,bc606373dd152e0585a444b5c77fa689
2698,"LINESTRING (-180785.421 27827.886, -180782.894...",a131b466de5793f858b5dec42fbbb4a9,50,1,bc606373dd152e0585a444b5c77fa689
3803,"LINESTRING (422429.667 -582166.415, 422282.476...",fe4aab1717eca5a2935c32c85a35a5bf,1,0,2e7565e0f7520c62b8210f9ba512e33e


In [84]:
lookback_xwalk = assemble_hqta_points.get_lookback_agency_crosswalk(published_operators_dict, lookback_trips_ix)

In [86]:
major_stop_bus

,schedule_gtfs_dataset_key_primary,schedule_gtfs_dataset_key_secondary,stop_id,geometry,hqta_type,iteration
0,0bcbcc353367cba8ba4134764abe8ec1,0666caf3ec1ecc96b74f4477ee4bc939,305,"POLYGON ((163919.518 -428467.431, 163919.518 -...",major_stop_bus,0
1,0bcbcc353367cba8ba4134764abe8ec1,0666caf3ec1ecc96b74f4477ee4bc939,280,"POLYGON ((163687.971 -428390.172, 163687.971 -...",major_stop_bus,0
2,0bcbcc353367cba8ba4134764abe8ec1,0666caf3ec1ecc96b74f4477ee4bc939,289,"POLYGON ((163941.315 -428498.871, 163941.315 -...",major_stop_bus,0
3,0bcbcc353367cba8ba4134764abe8ec1,0666caf3ec1ecc96b74f4477ee4bc939,279,"POLYGON ((163749.474 -428409.337, 163749.474 -...",major_stop_bus,0
4,0666caf3ec1ecc96b74f4477ee4bc939,0666caf3ec1ecc96b74f4477ee4bc939,9815,"POLYGON ((163709.487 -428386.666, 163709.487 -...",major_stop_bus,0
...,...,...,...,...,...,...
14266,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1677,"POLYGON ((-119492.529 66007.706, -119492.529 6...",major_stop_bus,0
14267,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1646,"POLYGON ((-119510.352 66049.990, -119510.352 6...",major_stop_bus,0
14268,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1885,"POLYGON ((-123277.194 55436.140, -123277.194 5...",major_stop_bus,0
14269,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,4022,"POLYGON ((-122079.290 51275.584, -122079.290 5...",major_stop_bus,0


In [87]:
major_stop_bus.query("schedule_gtfs_dataset_key_primary.isin(@lookback_xwalk.schedule_gtfs_dataset_key)")

,schedule_gtfs_dataset_key_primary,schedule_gtfs_dataset_key_secondary,stop_id,geometry,hqta_type,iteration
3632,227b598f6f0f7cdb9a39f799cffa7352,a02018fa11c962bcd3e07ab7f4516a63,5252,"POLYGON ((145099.754 -437703.806, 145099.754 -...",major_stop_bus,0
3633,227b598f6f0f7cdb9a39f799cffa7352,a02018fa11c962bcd3e07ab7f4516a63,5255,"POLYGON ((145123.725 -437718.723, 145123.725 -...",major_stop_bus,0
3658,227b598f6f0f7cdb9a39f799cffa7352,c2630c44a1f2703ec651596e6358d4f4,5252,"POLYGON ((145099.754 -437703.806, 145099.754 -...",major_stop_bus,0
3659,227b598f6f0f7cdb9a39f799cffa7352,c2630c44a1f2703ec651596e6358d4f4,5255,"POLYGON ((145123.725 -437718.723, 145123.725 -...",major_stop_bus,0
3734,227b598f6f0f7cdb9a39f799cffa7352,0666caf3ec1ecc96b74f4477ee4bc939,5252,"POLYGON ((145099.754 -437703.806, 145099.754 -...",major_stop_bus,0
...,...,...,...,...,...,...
14266,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1677,"POLYGON ((-119492.529 66007.706, -119492.529 6...",major_stop_bus,0
14267,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1646,"POLYGON ((-119510.352 66049.990, -119510.352 6...",major_stop_bus,0
14268,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1885,"POLYGON ((-123277.194 55436.140, -123277.194 5...",major_stop_bus,0
14269,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,4022,"POLYGON ((-122079.290 51275.584, -122079.290 5...",major_stop_bus,0


In [108]:
major_stop_bus_testing.query("schedule_gtfs_dataset_key_primary.isin(@lookback_xwalk.schedule_gtfs_dataset_key)")

,schedule_gtfs_dataset_key_primary,schedule_gtfs_dataset_key_secondary,stop_id,geometry,hqta_type,iteration
3006,227b598f6f0f7cdb9a39f799cffa7352,0666caf3ec1ecc96b74f4477ee4bc939,5252,"POLYGON ((144296.023 -437743.291, 144292.148 -...",major_stop_bus,4
3007,227b598f6f0f7cdb9a39f799cffa7352,0666caf3ec1ecc96b74f4477ee4bc939,5255,"POLYGON ((144319.994 -437758.208, 144316.120 -...",major_stop_bus,4
3026,227b598f6f0f7cdb9a39f799cffa7352,4b317fc27dde351e12253d46cedd8df0,5252,"POLYGON ((144296.023 -437743.291, 144292.148 -...",major_stop_bus,4
3027,227b598f6f0f7cdb9a39f799cffa7352,4b317fc27dde351e12253d46cedd8df0,5255,"POLYGON ((144319.994 -437758.208, 144316.120 -...",major_stop_bus,4
3054,227b598f6f0f7cdb9a39f799cffa7352,a02018fa11c962bcd3e07ab7f4516a63,5252,"POLYGON ((144296.023 -437743.291, 144292.148 -...",major_stop_bus,4
...,...,...,...,...,...,...
13229,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1677,"POLYGON ((-120296.259 65968.221, -120300.134 6...",major_stop_bus,4
13230,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1646,"POLYGON ((-120314.083 66010.505, -120317.957 6...",major_stop_bus,4
13231,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,1885,"POLYGON ((-124080.924 55396.655, -124084.799 5...",major_stop_bus,4
13232,98588c1726b50acbef430783ab0e734e,98588c1726b50acbef430783ab0e734e,4022,"POLYGON ((-122883.020 51236.099, -122886.895 5...",major_stop_bus,4
